In [1]:
from Idntimes import Idntimes
from datetime import timedelta, date, datetime
from IPython.display import clear_output

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)
        
obj = Idntimes()

In [6]:
end_date = date(2018, 8, 20)
start_date = date(2018, 8, 18)
init_page = 3

In [7]:
for single_date in daterange(start_date, end_date):
    clear_output()
    print(single_date.strftime("%Y-%m-%d"))
    obj.getAllBerita([], 'news', init_page, single_date.strftime("%Y-%m-%d"))
    init_page = 1

2018-08-19
page  1
https://www.idntimes.com/ajax/index?category=news&type=all&page=1&date=2018-08-19
https://www.idntimes.com/news/indonesia/vanny-rahman/6-kali-gempa-susulan-di-lombok-getaran-terasa-hingga-makassar
memasukkan berita id  1534611714
Insert berita  6 Kali Gempa Susulan di Lombok, Getaran Terasa Hingga Makassar
masuk
https://www.idntimes.com/news/indonesia/rochmanudin-wijaya/akibat-gempa-lombok-wisatawan-di-bali-berhamburan-ke-luar-hotel
memasukkan berita id  1534611722
Insert berita  Akibat Gempa Lombok, Wisatawan di Bali Berhamburan ke Luar Hotel
masuk
https://www.idntimes.com/news/indonesia/indianamalia/satu-orang-meninggal-akibat-gempa-65-sr-di-lombok-tadi-siang
memasukkan berita id  1534611713
Insert berita  Satu Orang Meninggal Akibat Gempa 6,5 SR di Lombok Tadi Siang
masuk
https://www.idntimes.com/news/indonesia/ardiansyah-fajar/gempa-7-sr-guncang-lombok-listrik-langsung-padam
memasukkan berita id  1534611705
Insert berita  Gempa 7 SR Guncang Lombok, Listrik Langsu

masuk
https://www.idntimes.com/news/indonesia/indianamalia/pengamat-terorisme-pawai-anak-tk-probolinggo-jangan-dianggap-remeh
memasukkan berita id  1534611719
Insert berita  Pengamat Terorisme: Pawai Anak TK Probolinggo Jangan Dianggap Remeh
masuk
https://www.idntimes.com/news/indonesia/dwifantya-aquina/foto-para-menteri-antre-selfie-bareng-joni-bocah-heroik-dari-ntt
memasukkan berita id  1534611721
Insert berita  FOTO: Para Menteri Antre Selfie Bareng Joni, Bocah Heroik dari NTT
masuk
https://www.idntimes.com/news/indonesia/dwifantya-aquina/jokowi-sambut-hangat-perdamaian-korea-selatan-dan-korea-utara
memasukkan berita id  1534611718
Insert berita  Jokowi Sambut Hangat Perdamaian Korea Selatan dan Korea Utara
masuk
https://www.idntimes.com/news/indonesia/masdalena-napitupulu/5-fakta-kofi-annan-dari-keluarga-elite-hingga-nobel-perdamaian
memasukkan berita id  1534611723
Insert berita  5 Fakta Kofi Annan, dari Keluarga Elite hingga Nobel Perdamaian
masuk
https://www.idntimes.com/news/in